In [ ]:
import sys; sys.path.append('../../')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../../../examples/florin/20190102_162258_meshID_e68521af-eaf1-40eb-8faa-e1b673097eee.obj', 8)
#driver=166
driver=l.centralJoint()

#mat = elastic_rods.RodMaterial('+', 2000, 0.3, [5, 5, 0.7, 0.7], stiffAxis=elastic_rods.StiffAxis.D1)
mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [12, 8])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l,  width=1800, height=1024)
view.setCameraParams(((-0.8791632874589965, 2.28378379944045, -0.8241729652214953),
 (-0.08744793312122603, -0.24477419166640266, -0.965628527999708),
 (0.23892776086999715, 0.21662971777148163, 0.740547140136883)))
view.show()

In [ ]:
l.saveVisualizationGeometry('deployed_meshID_bdb78930-da3e-4a28-98f5-89c2ee17876c.obj')
#l.writeLinkageDebugData('')

In [ ]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_meshID_1a219f3e-c853-4985-ad5f-f13b369c0e63.txt', zeroBasedIndexing=True)

In [ ]:
np.savetxt('restlen_meshID_1a219f3e-c853-4985-ad5f-f13b369c0e63.txt',l.getPerSegmentRestLength())

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, 2 * np.pi/3, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);
open_linkage(l, driver, np.deg2rad(90) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);